<a href="https://colab.research.google.com/github/supermarios77/DoodleIt/blob/main/DoodleIt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>